In [1]:
import numpy as np
import matplotlib.pyplot as plt
import nbimporter
import pandas as pd

import central_system as cs
import analyze_control as ac
import read_load as rl

import report_study_case
import report_house_spec
import report_general_property

Importing Jupyter notebook from report_study_case.ipynb
Importing Jupyter notebook from report_house_spec.ipynb
Importing Jupyter notebook from report_general_property.ipynb


# 居住者の使い方

本モジュールでは負荷計算等の与条件となる居住者の使い方について示します。  

## 0. 検討ケース

試算するケースを取得します。

In [2]:
case = report_study_case.get_case()
case

{'region': 6,
 'house': {'a_mr': 29.81,
  'a_or': 51.34,
  'a_a': 120.08,
  'r_env': 2.9555555555555557,
  'insulation': 'h11',
  'solar_gain': 'middle'},
 'system': {'default_heat_source_spec': True,
  'v_hs_rtd_h': 1800.0,
  'v_hs_rtd_c': 1800.0,
  'is_duct_insulated': True,
  'vav_system': False,
  'cap_rtd_h': None,
  'cap_rtd_c': None}}

## 1. 外部モジュールの読み込み

### 住宅の仕様

外部モジュール report_external_conditions を参照のこと。  

$r_{env}$：床面積の合計に対する外皮の部位の面積の合計の比  
$A_A$：床面積の合計, m<sup>2</sup>  
$A_{MR}$：主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$：その他の居室の床面積, m<sup>2</sup>  
$A_{NR}$：非居室の床面積, m<sup>2</sup>  
$A_{HCZ,R,i}$：標準住戸における暖冷房区画 $i$ の床面積, m<sup>2</sup>  
$A_{HCZ,i}$：暖冷房区画 $i$ の床面積, m<sup>2</sup>  
$A_{prt,i}$：暖冷房区画 $i$ から見た非暖冷房空間の間仕切りの面積, m<sup>2</sup>  
$U_{prt}$：間仕切りの熱貫流率, W/m<sup>2</sup>K  
$Q$：熱損失係数, W/m<sup>2</sup>K  
$\mu_H$：暖房期の日射取得係数, ((W/m<sup>2</sup>)/(W/m<sup>2</sup>))  
$\mu_C$：冷房期の日射取得係数, ((W/m<sup>2</sup>)/(W/m<sup>2</sup>))  
$V_{vent,i}$：暖冷房区画 $i$ の機械換気量, m<sup>3</sup>/h  

In [3]:
r_env, a_a, a_mr, a_or, a_nr, a_hcz, a_prt, u_prt, q, mu_h, mu_c, v_vent = \
    report_house_spec.get_house_spec(case['region'], case['house'])
r_env, a_a, a_mr, a_or, a_nr, a_hcz, a_prt, u_prt, q, mu_h, mu_c, v_vent

(2.9555555555555557,
 120.08,
 29.81,
 51.34,
 38.92999999999999,
 array([29.81, 16.56, 13.25, 10.76, 10.77,  3.31,  1.66,  3.31, 13.25,
         4.97, 10.77,  1.66]),
 array([37.99636619, 27.7187943 , 22.17838312, 18.01052093, 18.02725933]),
 2.1739130434782608,
 2.7,
 0.101,
 0.062,
 array([60., 20., 40., 20., 20.]))

### 物性値

外部モジュール report_general_property を参照のこと。

$\rho$：空気の密度, kg/m<sup>3</sup>  
$c$：空気の比熱, J/kgK  
$L_{wtr}$：水の蒸発潜熱, kJ/kg  
暖冷房期間

In [4]:
rho, c, l_wtr, calender = report_general_property.get_general_property()
rho, c, l_wtr, calender

(1.2,
 1006.0,
 2434.5296000000003,
 array(['休日', '休日', '休日', ..., '休日', '休日', '休日'], dtype=object))

## 2. 暖冷房期間

暖冷房期間は地域に応じて以下のように定まるとする。

| 地域 | 暖房期間 | 冷房期間 |
| --- | --- | --- |
| 1地域 | 9/24～6/7 | 7/10～8/31 |
| 2地域 | 9/26～6/4 | 7/15～8/31 |
| 3地域 | 9/30～5/31 | 7/10～8/31 |
| 4地域 | 10/1～5/30 | 7/10～8/31 |
| 5地域 | 10/10～5/15 | 7/6～8/31 |
| 6地域 | 11/4～4/21 | 5/30～9/23 |
| 7地域 | 11/26～3/27 | 5/15～10/13 |
| 8地域 | なし | 3/25～12/14 |

In [5]:
heating_period, cooling_period = cs.get_heating_and_cooling_schedule(case['region'])
heating_period, cooling_period

(array([ True,  True,  True, ...,  True,  True,  True]),
 array([False, False, False, ..., False, False, False]))

### 3.  在室人数スケジュール

$$ n_{p,d,t} = n_{p,MR,d,t} + n_{p,OR,d,t} + n_{p,NR,d,t} $$  
$$ n_{p,MR,d,t} = n_{s,p,MR,d,t} \frac{ A_{MR} }{ 29.81 } $$  
$$ n_{p,OR,d,t} = n_{s,p,OR,d,t} \frac{ A_{OR} }{ 51.34 } $$  
$$ n_{p,NR,d,t} = n_{s,p,NR,d,t} \frac{ A_{NR} }{ 38.93 } $$  

$n_{p,d,t}$：日付$d$の時刻$t$における在室人数  
$n_{p,MR,d,t}$：日付$d$の時刻$t$における主たる居室の在室人数  
$n_{p,MR,d,t}$：日付$d$の時刻$t$におけるその他の居室の在室人数  
$n_{p,MR,d,t}$：日付$d$の時刻$t$における非居室の在室人数  
$n_{s,p,MR,d,t}$：日付$d$の時刻$t$における標準住戸の主たる居室の在室人数  
$n_{s,p,MR,d,t}$：日付$d$の時刻$t$における標準住戸のその他の居室の在室人数  
$n_{s,p,MR,d,t}$：日付$d$の時刻$t$における標準住戸の非居室の在室人数  
$A_{MR}$：主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$：その他居室の床面積, m<sup>2</sup>  
$A_{NR}$：非居室の床面積, m<sup>2</sup>  

標準住戸における在室人数（人）

平日

| 居室の種類 | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 | 16 | 17 | 18 | 19 | 20 | 21 | 22 | 23 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 主たる居室 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 2 | 1 | 1 | 0 | 0 | 1 | 1 | 0 | 0 | 1 | 2 | 2 | 3 | 3 | 2 | 1 | 1 |
| その他の居室 | 4 | 4 | 4 | 4 | 4 | 4 | 3 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 1 | 2 | 3 |
| 非居室 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |

休日

| 居室の種類 | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 | 16 | 17 | 18 | 19 | 20 | 21 | 22 | 23 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 主たる居室 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 3 | 2 | 2 | 2 | 2 | 1 | 0 | 0 | 2 | 3 | 3 | 4 | 2 | 2 | 1 | 0 |
| その他の居室 | 4 | 4 | 4 | 4 | 4 | 4 | 4 | 3 | 1 | 2 | 2 | 2 | 1 | 0 | 0 | 0 | 1 | 1 | 1 | 0 | 2 | 2 | 2 | 3 |
| 非居室 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |

In [11]:
def get_n_p(a_mr, a_or, a_nr, calender):
    
    n_p_mr_wd = np.array([0, 0, 0, 0, 0, 0, 1, 2, 1, 1, 0, 0, 1, 1, 0, 0, 1, 2, 2, 3, 3, 2, 1, 1]) * a_mr / 29.81
    n_p_or_wd = np.array([4, 4, 4, 4, 4, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 2, 3]) * a_or / 51.34
    n_p_nr_wd = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) * a_nr / 38.93
    n_p_mr_hd = np.array([0, 0, 0, 0, 0, 0, 0, 0, 3, 2, 2, 2, 2, 1, 0, 0, 2, 3, 3, 4, 2, 2, 1, 0]) * a_mr / 29.81
    n_p_or_hd = np.array([4, 4, 4, 4, 4, 4, 4, 3, 1, 2, 2, 2, 1, 0, 0, 0, 1, 1, 1, 0, 2, 2, 2, 3]) * a_or / 51.34
    n_p_nr_hd = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) * a_nr / 38.93
    
    n_p_mr = np.tile(n_p_mr_wd, 365) * (calender == '平日') + np.tile(n_p_mr_wd, 365) * (calender == '休日')
    n_p_or = np.tile(n_p_or_wd, 365) * (calender == '平日') + np.tile(n_p_or_wd, 365) * (calender == '休日')
    n_p_nr = np.tile(n_p_nr_wd, 365) * (calender == '平日') + np.tile(n_p_nr_wd, 365) * (calender == '休日')
    
    n_p = n_p_mr + n_p_or + n_p_nr
    
    return n_p, n_p_mr, n_p_or, n_p_nr

n_p, n_p_mr, n_p_or, n_p_nr = get_n_p(a_mr, a_or, a_nr, calender)
n_p, n_p_mr, n_p_or, n_p_nr

(array([4., 4., 4., ..., 3., 3., 4.]),
 array([0., 0., 0., ..., 2., 1., 1.]),
 array([4., 4., 4., ..., 1., 2., 3.]),
 array([0., 0., 0., ..., 0., 0., 0.]))

In [10]:
def get_occupant_usage(region):

    heating_period, cooling_period = cs.get_heating_and_cooling_schedule(region)
    
    n_p_mr, n_p_or, n_p_nr = get_n_p(a_mr, a_or, a_nr, calender)
    
    return heating_period, cooling_period, n_p_mr, n_p_or, n_p_nr